# Import common library

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import os

In [2]:
os.chdir('C:\\Users\\user\\Documents\\Machine-Learning\\01. Introduction to Machine Learning\\Data')

# Week 1 - Introduction to Machine Learning

# Week 2 - Linear Regression (Chapter 3)

## using statsmodel

In [3]:
import statsmodels.api as sm
boston = pd.read_csv('boston.csv')
boston.head()

,crim,zn,indus,chas,nox,rm,age,dis,rad,tax,ptratio,black,lstat,medv
0,0.00632,18.0,2.31,0,0.538,6.575,65.2,4.0900,1,296,15.3,396.90,4.98,24.0
1,0.02731,0.0,7.07,0,0.469,6.421,78.9,4.9671,2,242,17.8,396.90,9.14,21.6
2,0.02729,0.0,7.07,0,0.469,7.185,61.1,4.9671,2,242,17.8,392.83,4.03,34.7
3,0.03237,0.0,2.18,0,0.458,6.998,45.8,6.0622,3,222,18.7,394.63,2.94,33.4
4,0.06905,0.0,2.18,0,0.458,7.147,54.2,6.0622,3,222,18.7,396.90,5.33,36.2


In [4]:
x = pd.DataFrame({'intercept':np.ones(boston.shape[0]),
                  'lstat':boston['lstat']})
type(x)

pandas.core.frame.DataFrame

In [22]:
# ways to constant term to predictor variable
x1 = sm.add_constant(boston['lstat'])

x2 = pd.DataFrame({'const': np.ones(boston['lstat'].shape[0]), 'lstat': boston.lstat})
x2

,const,lstat
0,1.0,4.98
1,1.0,9.14
2,1.0,4.03
3,1.0,2.94
4,1.0,5.33
...,...,...
501,1.0,9.67
502,1.0,9.08
503,1.0,5.64
504,1.0,6.48


In [23]:
model = sm.OLS(boston['medv'], x2).fit()
model.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                   medv   R-squared:                       0.544
Model:                            OLS   Adj. R-squared:                  0.543
Method:                 Least Squares   F-statistic:                     601.6
Date:                Wed, 23 Apr 2025   Prob (F-statistic):           5.08e-88
Time:                        07:19:38   Log-Likelihood:                -1641.5
No. Observations:                 506   AIC:                             3287.
Df Residuals:                     504   BIC:                             3295.
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const         34.5538      0.563     61.415      0.000      33.448      35.659
lstat         -0.9500      0.039    -24.528      0.000      -1.026      -0.874
==============================================================================
Omnibus:                      137.043   Durbin-Watson:                   0.892
Prob(Omnibus):                  0.000   Jarque-Bera (JB):              291.373
Skew:                           1.453   Prob(JB):                     5.36e-64
Kurtosis:                       5.319   Cond. No.                         29.7
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

## Using scikit-learn

# Week 3 - Classification (Chapter 4)

# Week 4 - Resampling Methods (Chapter 5)

# Week 5 - Linear Model Selection (Chapter 6)

### Forward Selection

In [6]:
hitters_df = pd.read_csv('01. Introduction to Machine Learning/Data/hitters.csv')
hitters_df.head()

FileNotFoundError: [Errno 2] No such file or directory: '01. Introduction to Machine Learning/Data/hitters.csv'

In [7]:
os.getcwd()

'C:\\Users\\user\\Documents\\Machine-Learning\\01. Introduction to Machine Learning\\Data'

In [8]:
# calculate the number of missing rows in Salary column
hitters_df['Salary'].isnull().sum()

NameError: name 'hitters_df' is not defined

In [9]:
hitters_df.shape

NameError: name 'hitters_df' is not defined

In [10]:
# remove any rows that contain missing values, along with the player names
hitters_df_clean = hitters_df.dropna().drop('Unnamed: 0', axis=1)
hitters_df_clean.shape

NameError: name 'hitters_df' is not defined

In [11]:
def nCp(sigma2, estimator, X, Y):
    n, p = X.shape
    Yhat = esimator.predict(X)
    RSS = np.sum((Y - Yhat) ** 2)
    return -(RSS + 2 * p * sigma2) / n

In [12]:
n = hitters_df_clean.shape
type(n)

NameError: name 'hitters_df_clean' is not defined

In [13]:
# one-hot encoding categorical data to convert into numerical data
dummies = pd.get_dummies(hitters_df_clean[['League', 'Division', 'NewLeague']])
dummies.head(10)

NameError: name 'hitters_df_clean' is not defined

In [14]:
# drop column with independent variable (salary) and columns for which we created dummy variable (league, division, new league)
y = hitters_df_clean.Salary
x_ = hitters_df_clean.drop(['Salary', 'League', 'Division', 'NewLeague'], axis=1)
x = pd.concat([x_, dummies[['League_N', 'Division_W', 'NewLeague_N']]], axis=1)
x

NameError: name 'hitters_df_clean' is not defined

In [15]:
import statsmodels.api as sm
sm.OLS(hitters_df_clean['Salary'], hitters_df_clean['Hits']).fit()

NameError: name 'hitters_df_clean' is not defined

In [16]:
from sklearn.linear_model import LinearRegression
lr = LinearRegression()
model = lr.fit(hitters_df_clean['Hits'].reshape(-1,1), hitters_df_clean['Salary'])
model.summary

NameError: name 'hitters_df_clean' is not defined

In [ ]:
x = [10,23,32]
x.astype(float)

# Week 6 - Nonlinear models (Chapter 7)

<h1> Trial and Error</h1>

In [17]:
marketing = pd.read_csv('marketing.csv')

FileNotFoundError: [Errno 2] No such file or directory: 'marketing.csv'

In [ ]:
os.chdir('C:\\Users\\user\\Desktop')

In [18]:
import statsmodels.api as sm
sm.OLS

statsmodels.regression.linear_model.OLS